# Group Work Project - M7

June 27, 2021

- **Course**: MScFE 620 Discrete-time Stochastic Processes
- **Professors**:
    - Sergio Garcia
    - Ivan Blanko
- **Class**: C21-S1
- Group 2, with members:
    - Juan Antonio Vargas Lopez; juanvarl@hotmail.com
    - Kanittha Setthapitayakul; kanittha.se@hotmail.com
    - Loc Nguyen; vinhloc30796@gmail.com

### 1. First, consider the case of an American Call option with underlying stock trading at \\$95\, Strike price of \\$90 and 1 year maturity. Use the Binomial model with the parameters r=0 and 5 steps in the pricing process. Additionally, set the upper movement of the price (u) to an expression that will depend on the number on your group:

<center>$u=(1.1+\frac{Group Number}{100})$

And define  <center> $d=\frac{1}{u}$

Consider American option

In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True) # prettify numpy
import pandas as pd
pd.set_option('precision', 4) # prettify pandas

Define initial input parameters as below:

In [2]:
# Set starting price at $95
s0 = 95
# Set strike price at $90
k = 90
r = 0
n = 5

In [3]:
# Calculate u, d, P_star
group_number = 2
u = 1.10 + group_number/100; d = 1/u
price_u = (1 - d) / (u - d)
price_d = 1-price_u

print('pu='+str(price_u))
print('pd='+str(price_d))

pu=0.4716981132075471
pd=0.5283018867924529


Create Binomial tree regarding to initial parameters:

In [4]:
def stocktree(n,s0,u,d): 
    n_stocktree = np.zeros((n+1,n+1))
    for j in range(n+1):
        for i in range(j+1):
            n_stocktree[i][j] = s0*(u**i)*(d**(j-i))
    return n_stocktree

In [5]:
step5_stocktree = stocktree(n,s0,u,d)
step5_stocktree

array([[ 95.    ,  84.8214,  75.7334,  67.6191,  60.3742,  53.9056],
       [  0.    , 106.4   ,  95.    ,  84.8214,  75.7334,  67.6191],
       [  0.    ,   0.    , 119.168 , 106.4   ,  95.    ,  84.8214],
       [  0.    ,   0.    ,   0.    , 133.4682, 119.168 , 106.4   ],
       [  0.    ,   0.    ,   0.    ,   0.    , 149.4843, 133.4682],
       [  0.    ,   0.    ,   0.    ,   0.    ,   0.    , 167.4225]])

Create function to calculate European option price in order to compare price between American and European option price

In [6]:
def european_payofftree(bi_tree,n,k,iscall):
    payoff_tree = np.zeros((n+1,n+1))
    iopt = 0
    if iscall:
        iopt = 1
    else:
        iopt = -1
    for i in range(n+1):
        payoff_tree[i][n] = np.maximum(0, iopt * (bi_tree[i][n]-K))
    return payoff_tree

def probtree(n,pu,pd):
    import math as m
    prob_tree = np.zeros((n+1,n+1))
    for j in range(n+1):
        for i in range(j+1):
            prob_tree[i][j] = m.factorial(j) / (m.factorial(i) * m.factorial(j-i))* pu**i * pd**(j-i)
    return prob_tree
def european_price(eu_payofftree,prob_tree,n):
    finalpayoff = 0.0
    for i in range(n+1):
        finalpayoff = finalpayoff + eu_payofftree[i][n] * prob_tree[i][n]
        optionprice = finalpayoff
    return finalpayoff

Calculate option payoff at each nodes

<center>$g_{Call}=max(S_0-K,0)$

<center>$g_{Put}=max(K-S_0,0)$

In [7]:
def american_payofftree(bi_tree,n,k,iscall):
    payoff_tree = np.zeros((n+1,n+1))
    # Get payoffs at each node
    if iscall:
        iopt = 1
    else:
        iopt = -1
    for j in range(n+1):
        for i in range(j+1):
            payoff_tree[i][j] = np.maximum(0,iopt*(bi_tree[i][j]-k))
    return payoff_tree

Calculate value of option at each nodes from backward recursion of option payoff:

<center>$v_{k} = max(\frac{1}{1+r}[g_{u}(p_{u})+g_{d}(p_{d})],g_{k})$

In [8]:
def american_optprctree(payoff_tree,pu,pd,K,iscall,r):
    opt_prc_tree = payoff_tree
    if iscall:
        iopt = 1
    else:
        iopt = -1
    for j in range(n-1,-1,-1): 
        for i in range(j+1):
            opt_prc_tree[i][j] = (1/(1+r))*(pu * payoff_tree[i+1][j+1]+ pd * payoff_tree[i][j+1])
            opt_prc_tree[i][j] = np.maximum(payoff_tree[i][j],opt_prc_tree[i][j])
    return opt_prc_tree     

In [9]:
def e_americanoptprc(prob_tree,us_pricetree,n):
    for j in range(n+1):
        expect_price = 0
        for i in range(j+1):
            expect_price = expect_price + us_pricetree[i][j] * prob_tree[i][j]
        print('step '+str(j)+': '+str(expect_price))

#### a. In this setting, find the value of the derivative at each node.

#### ANSWER

In [10]:
iscall = True
call_us_payofftree = american_payofftree(step5_stocktree,n,k,iscall)
call_us_payofftree

array([[ 5.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    , 16.4   ,  5.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    , 29.168 , 16.4   ,  5.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    , 43.4682, 29.168 , 16.4   ],
       [ 0.    ,  0.    ,  0.    ,  0.    , 59.4843, 43.4682],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , 77.4225]])

In [11]:
call_us_optprctree = american_optprctree(call_us_payofftree,price_u,price_d,k,iscall,r)
call_us_optprctree

array([[12.2963,  5.7892,  1.7212,  0.    ,  0.    ,  0.    ],
       [ 0.    , 19.5842, 10.3454,  3.649 ,  0.    ,  0.    ],
       [ 0.    ,  0.    , 29.9316, 17.8454,  7.7358,  0.    ],
       [ 0.    ,  0.    ,  0.    , 43.4682, 29.168 , 16.4   ],
       [ 0.    ,  0.    ,  0.    ,  0.    , 59.4843, 43.4682],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , 77.4225]])

In [12]:
print('American call option price is '+str(call_us_optprctree[0][0]))

American call option price is 12.296267645333053


#### b. Is there any point time where we, as buyers of the option, benefit from early exercise?

#### Answer: 

From above, we can see that there is a possibility that the option price is more than different between Spot price and Strike price. Thus, in this case, there is benefit from early exercise.

#### c. Explain whether your previous answer is (or is not) always the case.

#### ANSWER:

For an american call option, early excercise is never benefit to buyer in case the interest rate (r) and the time value applied . For example, if r = 10%, the option price will be

In [13]:
call_us_optprctree = american_optprctree(call_us_payofftree,price_u,price_d,k,iscall,0.1)
call_us_optprctree

array([[ 7.635 ,  3.9541,  1.2932,  0.    ,  0.    ,  0.    ],
       [ 0.    , 13.3762,  7.7726,  3.0157,  0.    ,  0.    ],
       [ 0.    ,  0.    , 22.488 , 14.7482,  7.0326,  0.    ],
       [ 0.    ,  0.    ,  0.    , 35.9241, 26.5164, 16.4   ],
       [ 0.    ,  0.    ,  0.    ,  0.    , 54.0767, 43.4682],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , 77.4225]])

We can see that when the spot price more than strike price the option value is lower than its payoff. However, the american call option might be benefit to buyer especially with dividend paying stock.

### 2. Consider the same parameters and setting as in (1), but now for pricing an American Put Option.

#### a. Find the value of the derivative at each node.

#### ANSWER:

In [14]:
iscall = False
put_us_payofftree = american_payofftree(step5_stocktree,n,k,iscall)
put_us_payofftree

array([[ 0.    ,  5.1786, 14.2666, 22.3809, 29.6258, 36.0944],
       [ 0.    ,  0.    ,  0.    ,  5.1786, 14.2666, 22.3809],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  5.1786],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]])

In [15]:
put_us_optprctree = american_optprctree(put_us_payofftree,price_u,price_d,k,iscall,r)
put_us_optprctree

array([[ 7.2963, 10.9678, 15.9878, 22.3809, 29.6258, 36.0944],
       [ 0.    ,  3.1842,  5.3454,  8.8276, 14.2666, 22.3809],
       [ 0.    ,  0.    ,  0.7636,  1.4454,  2.7358,  5.1786],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]])

In [16]:
print('American put option price is '+str(put_us_optprctree[0][0]))

American put option price is 7.2962676453330495


#### b. Is there any point time where we, as buyers of the option, benefit from early exercise?

#### ANSWER:

From above, we can see that there is a possibility that the option price is more than different between Spot price and Strike price. Thus, in this case, there is benefit from early exercise.

#### c. Explain whether your previous answer is (or is not) always the case.

#### ANSWER:

The American Put option will not be benefit to buyer to early exercise if the spot price is higher than the exercise boundary. 

### 3. Finally, modify your code accordingly in order to price an exotic option with European payoff. In this case we are going to price an Up-and-Out (UAO) European Call Option. The payoff at maturity (T) of such option is as follows:

$v(S_T)=(𝑆_T−𝐾)^+$ given $max_{[0,𝑇]}𝑆_t<𝐿$

where L is the barrier level such that, if trespassed, the value of the option becomes 0. 

Notice that the value of this option is path-dependent, not just on terminal value. The maturity of the option is 1 year, and is currently struck at-the-money (ATM).

The up-and-out barrier for the option is USD 130, while the current underlying stock price is USD  100. Consider the Binomial model under the same u, d, and r parameters from part (1) in this submission and 5 steps in the pricing process.

In [17]:
S2 = 100
L = 130
X2 = 100
# Group Number
group_number = 2
u = 1.10 + group_number/100; d = 1/u

In [18]:
def recursive_name(u,d,n):
    if n <=1:
        return [u,d]
    else:
        te = recursive_name(u,d,n-1)
        total_list = []
        for i2 in te:
            for i1 in [u,d]:
                total_list.append(i2+i1)
        return total_list

In [19]:
def cal_exotic_option(s,x,l,u,d,n,t,r):
    p = (1-d)/(u-d)
    p_inv = 1-p
    temp = np.ones((2**n,n+2))*s
    price_df = pd.DataFrame(temp, columns=['w']+[f'P({i})' for i in range(n+1)])
    price_df['w'] = recursive_name('u','d',n)
    dic = {}
    def cal_price_prob(a1,i1,u1,d1,s1,t1):
        te_str = a1['w'][:i1]
        u_count = te_str.count('u')
        d_count = te_str.count('d')
        if t1=='price':
            return s1*(u1**u_count)*(d1**d_count)
        elif t1=='prob':
            return (p**u_count)*(p_inv**d_count)
    for i in range(n+1):
        price_df[f'P({i})'] = price_df.apply(cal_price_prob,args=[i,u,d,s,'price'],axis=1)
    price_df['Prob'] = price_df.apply(cal_price_prob,args=[i,u,d,s,'prob'],axis=1)
    price_df['Value']= s
    if t=='UAO':
        price_df_temp = price_df[[f'P({i})' for i in range(n+1)]]
        for i in range(len(price_df.index)):
            ma = max(price_df_temp.loc[i].values)
            if ma > l:
                price_df['Value'].loc[i] = 0
            else:
                if price_df[f'P({n})'].loc[i] <= x:
                    price_df['Value'].loc[i] = 0
                else:
                    price_df['Value'].loc[i] = price_df[f'P({n})'].loc[i] - x
        price_df['Temp'] =price_df['Prob'] *price_df['Value']
        sum_UAO = sum(price_df["Temp"].values)
        price_df = price_df.drop(['Temp'],axis=1)
        dic['UAO'] = sum_UAO
    elif t=='UAI':
        price_df_temp = price_df[[f'P({i})' for i in range(n+1)]]
        for i in range(len(price_df.index)):
            ma = max(price_df_temp.loc[i].values)
            if ma < l:
                price_df['Value'].loc[i] = 0
            else:
                if price_df[f'P({n})'].loc[i] <= x:
                    price_df['Value'].loc[i] = 0
                else:
                    price_df['Value'].loc[i] = price_df[f'P({n})'].loc[i] - x
        price_df['Temp'] =price_df['Prob'] *price_df['Value']
        sum_UAI = sum(price_df["Temp"].values)
        price_df = price_df.drop(['Temp'],axis=1)
        dic['UAI'] = sum_UAI
    elif t=='normal':
        def cal_normal(a1):
            val = a1[f'P({n})']
            if val <= x:
                return 0
            else:
                return val - x 
        price_df['Value'] = price_df.apply(cal_normal,axis=1)
        price_df['Temp'] =price_df['Prob'] *price_df['Value']
        sum_normal = sum(price_df["Temp"].values)
        price_df = price_df.drop(['Temp'],axis=1)
        dic['normal'] = sum_normal

    print(f'The value of {t} European call option is: {dic[t]:.3f}$')
    print(f'The data table for {t} European call option is below:')
    return price_df

#### a. Find the value of this option.

#### ANSWER

In [20]:
cal_exotic_option(
    S2,X2,L,u,d,5,'UAO',0
)

The value of UAO European call option is: 3.164$
The data table for UAO European call option is below:


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,w,P(0),P(1),P(2),P(3),P(4),P(5),Prob,Value
0,uuuuu,100.0,112.0000,125.4400,140.4928,157.3519,176.2342,0.0234,0.0
1,uuuud,100.0,112.0000,125.4400,140.4928,157.3519,140.4928,0.0262,0.0
2,uuudu,100.0,112.0000,125.4400,140.4928,125.4400,140.4928,0.0262,0.0
3,uuudd,100.0,112.0000,125.4400,140.4928,125.4400,112.0000,0.0293,0.0
4,uuduu,100.0,112.0000,125.4400,112.0000,125.4400,140.4928,0.0262,0.0
5,uudud,100.0,112.0000,125.4400,112.0000,125.4400,112.0000,0.0293,12.0
6,uuddu,100.0,112.0000,125.4400,112.0000,100.0000,112.0000,0.0293,12.0
7,uuddd,100.0,112.0000,125.4400,112.0000,100.0000,89.2857,0.0328,0.0
8,uduuu,100.0,112.0000,100.0000,112.0000,125.4400,140.4928,0.0262,0.0
9,uduud,100.0,112.0000,100.0000,112.0000,125.4400,112.0000,0.0293,12.0


#### b. Which option is more expensive: the European call, or the UAO European call?<br>

#### ANSWER

The price of the normal option is more expensive than the UAO European call option 

$$(19.202 > 1.655)$$

In [21]:
cal_exotic_option(
    S2,X2,L,u,d,5,'normal',0
)

The value of normal European call option is: 10.591$
The data table for normal European call option is below:


,w,P(0),P(1),P(2),P(3),P(4),P(5),Prob,Value
0,uuuuu,100.0,112.0000,125.4400,140.4928,157.3519,176.2342,0.0234,76.2342
1,uuuud,100.0,112.0000,125.4400,140.4928,157.3519,140.4928,0.0262,40.4928
2,uuudu,100.0,112.0000,125.4400,140.4928,125.4400,140.4928,0.0262,40.4928
3,uuudd,100.0,112.0000,125.4400,140.4928,125.4400,112.0000,0.0293,12.0000
4,uuduu,100.0,112.0000,125.4400,112.0000,125.4400,140.4928,0.0262,40.4928
5,uudud,100.0,112.0000,125.4400,112.0000,125.4400,112.0000,0.0293,12.0000
6,uuddu,100.0,112.0000,125.4400,112.0000,100.0000,112.0000,0.0293,12.0000
7,uuddd,100.0,112.0000,125.4400,112.0000,100.0000,89.2857,0.0328,0.0000
8,uduuu,100.0,112.0000,100.0000,112.0000,125.4400,140.4928,0.0262,40.4928
9,uduud,100.0,112.0000,100.0000,112.0000,125.4400,112.0000,0.0293,12.0000


#### c. What is the advantage of the up-and-out European call option?

The advantage of the UAO call option is that, if we could estimate the variation range of a stock price with a good confidence level (95%), we could have the same payoff as normal European call option - despsite the UAO option being much cheaper than the normal European option.

#### d. There is another exotic option called an Up-and-In (UAI) European Call option... 

Without running a binomial tree, what is the price of UAI with the same strike and maturity as the European call, and the same strike, maturity, and barrier as the UAO. Assume the current stock price is below the barrier. (Hint: Think of a parity using the European call, UAI, and UAO).

#### ANSWER:

We have the following table of the prices of the options:

|Option|UAO|UAI|Normal|
|--|--|--|--|
|Price|3.155 |16.047 |19.202|

Thus, we can conclude that: 

$$Pr(UAO) + Pr(UAI) = Pr(Normal)$$

In [22]:
cal_exotic_option(
    S2,X2,L,u,d,5,'UAI',0
)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


The value of UAI European call option is: 7.427$
The data table for UAI European call option is below:


,w,P(0),P(1),P(2),P(3),P(4),P(5),Prob,Value
0,uuuuu,100.0,112.0000,125.4400,140.4928,157.3519,176.2342,0.0234,76.2342
1,uuuud,100.0,112.0000,125.4400,140.4928,157.3519,140.4928,0.0262,40.4928
2,uuudu,100.0,112.0000,125.4400,140.4928,125.4400,140.4928,0.0262,40.4928
3,uuudd,100.0,112.0000,125.4400,140.4928,125.4400,112.0000,0.0293,12.0000
4,uuduu,100.0,112.0000,125.4400,112.0000,125.4400,140.4928,0.0262,40.4928
5,uudud,100.0,112.0000,125.4400,112.0000,125.4400,112.0000,0.0293,0.0000
6,uuddu,100.0,112.0000,125.4400,112.0000,100.0000,112.0000,0.0293,0.0000
7,uuddd,100.0,112.0000,125.4400,112.0000,100.0000,89.2857,0.0328,0.0000
8,uduuu,100.0,112.0000,100.0000,112.0000,125.4400,140.4928,0.0262,40.4928
9,uduud,100.0,112.0000,100.0000,112.0000,125.4400,112.0000,0.0293,0.0000
